In [160]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [161]:
# 분류 모델
# 의사결정나무 분류 모델
from sklearn.tree import DecisionTreeClassifier

In [162]:
# 학습용, 평가용(test)용 데이터 구분하기 위한 패키지 가져오기
from sklearn.model_selection import train_test_split

In [163]:
# train / test(new) data 70:30 / 80:20
# train(train, validation) / test data(new data) (60:15) : 25

In [164]:
# 데이터 불러오기
path = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

In [165]:
raw = pd.read_csv(path)

df = raw.copy()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [166]:
df.shape
# 891개 데이터, 12개의 속성 정보

(891, 12)

In [167]:
df.info()
# 데이터 결측치(not-null) 유무, 데이터 유형

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [168]:
# 기초 요약 통계량
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [169]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [170]:
# 결측치 확인 밀 개수 세기
df['Age'].isnull().sum()

177

In [171]:
177 / 891 * 100
# 20% 정도 null. 제거 불가(업계 관행상 5%내만 가능) >> 대체

19.865319865319865

In [172]:
# 단순 대치법 : 평균
df['Age'].mean()

29.69911764705882

In [173]:
age_mean = df['Age'].mean()
df['Age'].fillna(age_mean, inplace=True)

In [174]:
df['Age'].isnull().sum()

0

In [175]:
# Embarked 컬럼
df['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [176]:
# 최빈값 대치
df['Embarked'].mode()
df['Embarked'].mode()[0]

'S'

In [177]:
df_mode = df['Embarked'].mode()[0]
df['Embarked'].fillna(df_mode,inplace=True)

In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [179]:
from sklearn.preprocessing import LabelEncoder
df['Embarked'] = LabelEncoder().fit_transform(df['Embarked'])
df['Embarked']

0      2
1      0
2      2
3      2
4      2
      ..
886    2
887    2
888    2
889    0
890    1
Name: Embarked, Length: 891, dtype: int64

In [180]:
# df['Sex'] >> 1,0 라벨인코딩(원핫인코딩)
from sklearn.preprocessing import LabelEncoder

df['Sex'] = LabelEncoder().fit_transform(df['Sex'])
df['Sex']

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [181]:
# 파생변수 ('SibSp','Parch' >> Family)
df['Family'] = df['SibSp'] + df['Parch']
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0,1
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,0


In [182]:
df.drop(columns=('Cabin'), inplace=True)

In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    int64  
 11  Family       891 non-null    int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 83.7+ KB


분석 데이터 셋 준비
- X : 독립변수(설명변수)
- y : 종속변수(목표변수)

In [184]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked', 'Family'],
      dtype='object')

In [185]:
df[['Pclass', 'Sex', 'Age','Fare', 'Embarked', 'Family']]

,Pclass,Sex,Age,Fare,Embarked,Family
0,3,1,22.000000,7.2500,2,1
1,1,0,38.000000,71.2833,0,1
2,3,0,26.000000,7.9250,2,0
3,1,0,35.000000,53.1000,2,1
4,3,1,35.000000,8.0500,2,0
...,...,...,...,...,...,...
886,2,1,27.000000,13.0000,2,0
887,1,0,19.000000,30.0000,2,0
888,3,0,29.699118,23.4500,2,3
889,1,1,26.000000,30.0000,0,0


In [186]:
df['Survived']

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [187]:
x = df[['Pclass', 'Sex', 'Age','Fare', 'Embarked', 'Family']]
y = df['Survived']

In [188]:
# 데이터 분할(8:2 분할)

x_train, x_test, y_train, y_test = \
train_test_split(x,y, test_size=0.2, random_state=42)

In [189]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(712, 6)
(179, 6)
(712,)
(179,)


In [190]:
# 모델링
# 의사결정나무 모델 가져오기

dt = DecisionTreeClassifier(random_state=42)

# 학습 수행 >> 모델 생성(dt model)
dt.fit(x_train, y_train) # 훈련

DecisionTreeClassifier(random_state=42)

In [191]:
# 학습이 완료된 dt model 가지고 test data로 예측 수행
y_pred = dt.predict(x_test)

In [192]:
# 우리의 목표 : 에러(error) 최소화
# >> y_pred과 y_real(y_test)의 차이가 0에 가깝게 만드는 것

y_pred

array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1])

In [193]:
print(y_test)

709    1
439    0
840    0
720    1
39     1
      ..
433    0
773    0
25     1
84     1
10     1
Name: Survived, Length: 179, dtype: int64


In [194]:
# 모델 성능 비교 - 정확도 측정
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
print(acc)

0.7541899441340782
